In [1]:
# @title Initialize
hf_token = "" # @param {"type":"string","placeholder":"Your HuggingFace access token"}

from pathlib import Path
Path("/content/images").mkdir(exist_ok=True)
Path("/content/ffmpeg").mkdir(exist_ok=True)

!git clone --branch cuda https://github.com/hhkcu/open-oasis.git

print("Installing dependencies")

%cd /content

!wget -q "$(curl -s -I https://github.com/cloudflare/cloudflared/releases/latest|grep location|cut -d" " -f 2|tr -d "\r"|sed s/tag/download/)/cloudflared-linux-amd64" -O /content/cloudflared
!chmod a+x /content/cloudflared
!wget "https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz" -O ffmpeg.tar.xz
!tar xf ffmpeg.tar.xz --strip-components=1 -C /content/ffmpeg

!pip install -U "huggingface-hub[cli]"
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install einops diffusers timm av websockets ffmpeg-python safetensors

%cd /content/open-oasis

print("Downloading models")

!huggingface-cli login --token $hf_token
!huggingface-cli download --local-dir . Etched/oasis-500m oasis500m.safetensors
!huggingface-cli download --local-dir . Etched/oasis-500m vit-l-20.safetensors


Cloning into 'open-oasis'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 200 (delta 82), reused 81 (delta 57), pack-reused 75 (from 1)
Receiving objects: 100% (200/200), 28.61 MiB | 12.82 MiB/s, done.
Resolving deltas: 100% (101/101), done.
Installing dependencies
/content
--2024-11-06 19:03:22--  https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz
Resolving johnvansickle.com (johnvansickle.com)... 107.180.57.212
Connecting to johnvansickle.com (johnvansickle.com)|107.180.57.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41888096 (40M) [application/x-xz]
Saving to: ‘ffmpeg.tar.xz’

ffmpeg.tar.xz       100%[===================>]  39.95M  12.5MB/s    in 3.2s    

2024-11-06 19:03:26 (12.5 MB/s) - ‘ffmpeg.tar.xz’ saved [41888096/41888096]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━

To insert a custom starting image, run the cell above to initialize everything, then place any images you want into the `images` folder.

After that, run the cell below.

In [22]:
# @title Parse Starting Images
import os, pathlib, ffmpeg, sys

crop_width=640
crop_height=360

imagesPath = "/content/images"
samplesPath = os.path.join("/content/open-oasis", "sample_data")
for fn in os.listdir(imagesPath):
  print(os.path.join(samplesPath, pathlib.Path(fn).stem+".mp4"))
  ffmpeg.input(os.path.join(imagesPath, fn),loop=1).output(os.path.join(samplesPath, pathlib.Path(fn).stem+".mp4"),vcodec="libx264",t=1,pix_fmt="yuv420p",vf=f'crop=iw:iw*9/16,scale={crop_width}:{crop_height}').run()

/content/open-oasis/sample_data/2024-11-04_20.07.59.mp4


Error: ffmpeg error (see stderr output for detail)

In [30]:
# @title Run Game
starting_img_name = "2024-11-04_20.07.59" # @param {"type":"string","placeholder":"Filename of the starting image"}
%cd /content
!./cloudflared tunnel --url http://127.0.0.1:17890 &>c.log &
!sleep 6
!grep "  https" c.log |cut -d " " -f 5
%cd /content/open-oasis
!STARTING_IMAGE_NAME=$starting_img_name python3 game.py


/content
https://fixes-maintain-shadows-tutorials.trycloudflare.com
/content/open-oasis
/content/open-oasis/game.py:229: DeprecationWarning: Application.make_handler(...) is deprecated, use AppRunner API instead
  handler = app.make_handler()
loading vae
torch.Size([25, 360, 640, 3])
Traceback (most recent call last):
  File "/content/open-oasis/game.py", line 406, in <module>
    x = sample(x, actions_tensor, ddim_noise_steps, ctx_max_noise_idx, model)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
  File "/content/open-oasis/game.py", line 315, in sample
    v = model(x_curr, t, actions_tensor)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
  File

In [29]:
# @title fix
!pkill cloudflared

In [28]:
# @title (for me) pull
%cd /content/open-oasis
!git pull

/content/open-oasis
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 501 bytes | 501.00 KiB/s, done.
From https://github.com/hhkcu/open-oasis
   bec0f93..c71352f  cuda       -> origin/cuda
Updating bec0f93..c71352f
Fast-forward
 game.py | 29 +++++++++++++++++++++++++++++
 1 file changed, 29 insertions(+)
